# The Asymmetric 2D Pendulum and Lissajous Curves

*By* Timothy A. V. Teatro 

A presentation for UOIT's annual Science Rendezvous

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%pylab

To model a 2D-pendulum, I'll divide it into two single-dimensional pendulums. Since we only need to work on one dimension at a time, let us consider Newton's equation of motion for a pendulum with drag in a single dimension:

$$
\ddot\theta - \frac{g}{r}\,\sin\theta + \frac{b}{m}\,\dot{\theta} = 0.
$$

The $\sin\theta$ term is the restoration term. It is the balance of tension and gravity coercing the pendulum pith back towards the resting position. The $\dot{\theta}$ term is the drag term. It represents parasitic losses from the air drag on the ball, and friction in the hinges. Notice that it is proportional to velocity: the faster things move, the faster energy is lost to the environment.

It is useful to represent the system as a vector valued function on a two-dimensional state space $\boldsymbol{x}\in\mathbf{R}$ where the first dimension is the angular displacement, $\theta$, and the second dimension is the angular speed $\dot{\theta} = \omega$. The vector equation for the system is

$$
\dot{\boldsymbol{x}} = \frac{\mathrm{d}}{\mathrm{d}t}\begin{bmatrix}\theta\\ \dot\theta\end{bmatrix} = \begin{bmatrix}\dot\theta\\ \frac{-g}{m\,r}\sin\theta + \frac{b}{m}\,\dot{\theta}\end{bmatrix} =: \boldsymbol{f}(\boldsymbol{x}),
$$

The energy of the pendulum will by the sum of Kinetic energy:

$$
  T = \frac{1}{2}\,m\,(r\,\dot{\theta})^2,
$$

and potential energy:

$$
  U = m\,g\,r\,(1-\cos\theta),
$$

so $E = T + U$.

In Python, we'll express all of this in a class containing no state except the parameters. The class should also have a method for converting angular to Cartesian coordinates, since that conversion will depend on the structure of the pendulum.

In [ ]:
class Pendulum1D:
    """ Just the dynamics of the pendulum. Constants related to pendulum
        design are stored here, but not the pendulum's state.
    """

    def __init__(self, r=1, m=.5, b=.2, g=9.807):
        self.r = r  # Radius, or length of string.
        self.m = m  # Pith mass
        self.b = b  # Drag loss coefficient
        self.g = g  # Acceleration due to gravity

    def dxdt(self, x, t):
        # State of pendulum: x ∈ R^2, contains angular position and speed.
        return np.array(
            [x[1], -9.8 * np.sin(x[0]) / self.r - self.b * x[1] / self.m])

    def energy(s, x):
        T = s.m * (s.r * x[1])**2 / 2.
        U = s.m * s.g * s.r * (1 - np.cos(x[0]))
        return T + U

    def proj_horiz(self, pX):
        """ The dynamics are performed on the angular coordinates. For plotting,
            we'll want to convert with some basic trigonometry.
                x = r * sin(theta)
                dx/dt = r * cos(theta) * d.theta/dt
        """
        return np.array(
            [self.r * np.sin(pX[0]), self.r * np.cos(pX[0]) * pX[1]])

The state of the simulation is contained in an object type, which aggregates the two pendulums. It also has a method for computing the total energy of the pendulum, as the sum of the individual pendulums.

In [ ]:
class PendulumState2D:
    """ The class aggregates dynamics for two 1D pendulums, simulating a 2D
        pendulum.
    """

    # yapf: disable
    def __init__(self, r=[1, 1], m=0.5, b=0.2, g=9.807, dt=0.1,  # As before
                 xx0=[0, 1],    # Initial state of x-pendulum
                 xy0=[0, -1]):  # Initial state of y-pendulum
        self.dt = dt
        self.px = Pendulum1D(r=r[0], m=m, b=b, g=g)
        self.py = Pendulum1D(r=r[1], m=m, b=b, g=g)

        # Initial conditions
        self.t = 0
        self.xx, self.xy = np.array(xx0), np.array(xy0)
        self.xx0, self.xy0 = np.array(xx0), np.array(xy0)
        self.E0 = self.energy(xx0, xy0)  # Starting energy
    # yapf: enable

    def reset(self):
        self.t = 0
        self.xx, self.xy = self.xx0, self.xy0

    def energy(self, xx=None, xy=None):
        if xx is None:
            xx, xy = self.xx, self.xy
        return self.px.energy(xx) + self.py.energy(xy)

    def step(self):
        self.xx = odeint(self.px.dxdt, self.xx, [0, self.dt])[-1]
        self.xy = odeint(self.py.dxdt, self.xy, [0, self.dt])[-1]
        self.t += self.dt

Now, define the initial conditions to be passed to the `PendulumState2D` initialiser:

In [ ]:
initial_contidionts = dict(r=[.78, .9], b=0.04, xx0=[1, -.2], xy0=[-.33, .1])

The action of stepping the simulation forward and returning the state can be conveniently expressed in a generator. A generator is an abstraction in Python that combines an interator and a coroutine. Rather than `return`ing from a generator, you merely `yield` which doesn't clear the stack frame, alloing the function to resume from the previous tate. The use of `yield` automatically causes the fucntion to return a generator object, which can be iterated over. We'll create one, and later, pass it to the antimation routine:

In [ ]:
def pendulum_sim_generator():
    """ A generator which steps the simulation and returns the simulated state.
    """
    state = PendulumState2D(**initial_contidionts, dt=0.03)

    while True:
        yield state
        state.step()

At this point, all of the simulation logic is handled, now we can go about the business of plotting and animating:

In [ ]:
    fig = plt.figure()
    ax = fig.add_subplot(
        111, aspect='equal', autoscale_on=False, xlim=(-1, 1), ylim=(-1, 1))
    fig.subplots_adjust(
        left=0.05, right=0.98, bottom=0.05, top=0.98, wspace=0.1)

    line, = ax.plot([], [], 'r-', lw=1)
    pith, = ax.plot([], [], 'o-', ms=15)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    energy_text = ax.text(0.02, 0.92, '', transform=ax.transAxes)

    XHistory = []
    YHistory = []

    def animate(sim_state):

        if np.abs(sim_state.energy()) <= 0.05 * sim_state.E0:
            sim_state.reset()
            XHistory.clear()
            YHistory.clear()
            return line, pith, time_text, energy_text

        XHistory.append(sim_state.xx[0])
        YHistory.append(sim_state.xy[0])
        pith.set_data(sim_state.xx[0], sim_state.xy[0])
        line.set_data(XHistory, YHistory)
        time_text.set_text('time = %.1f' % sim_state.t)
        energy_text.set_text('energy = %.2f J' % sim_state.energy())
        return line, pith, time_text, energy_text

    ani = animation.FuncAnimation(
        fig, animate, pendulum_sim_generator, blit=True, interval=10)
    mng = plt.get_current_fig_manager()
    mng.full_screen_toggle()
    plt.show()

Copyright (C) 2017 By Timothy A. V. Teatro. All rights reserved.